# Make Python Sing!

In [33]:
def note_to_frequency(note):
    """
    Specify the note as a letter note, optional modifier (b,bb,#,##), and octave number.
    """
    note_letter = note[0].upper()
    octave = int(note[-1])
    half_step_mod = 0
    if len(note) > 2:
        modifier = note[1:-1]
        half_step_mod = {
            "b":-1,
            "bb":-2,
            "#":1,
            "##":2
        }[modifier]

    # reference frequency is A4 = 440 Hz
    a4 = 440
    # calculate which octave we are in relative to a4
    octaves_from_a4 = octave - 4
    # calculate how many half steps this note letter is from A in the
    # same octave
    note_letter_distance = {
        "A": 0,
        "B": 2,
        "C": -9,
        "D": -7,
        "E": -5,
        "F": -4,
        "G": -2,
    }[note_letter]
    # calculate how many half steps away we are total
    total_half_steps = octaves_from_a4*12 + note_letter_distance + half_step_mod
    # calculate the frequency. For simplicity, just use equal-tempered tuning.
    frequency = a4*(2**(total_half_steps/12))
    return frequency
    
        

123.47082531403103

In [26]:
"D#3"[1:-1]

'#'